<a href="https://colab.research.google.com/github/G0nkly/pytorch_sandbox/blob/main/vit/paligemma/processing_paligemma.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from typing import Dict, List, Optional, Union, Tuple, Iterable
import numpy as np
from PIL import Image
import torch

In [2]:
IMAGE_STANDARD_MEAN = [0.5, 0.5, 0.5]
IMAGE_STANDARD_STD = [0.5, 0.5, 0.5]

In [3]:
# https://github.com/google-research/big_vision/blob/main/big_vision/configs/proj/paligemma/README.md

# https://huggingface.co/docs/transformers/model_doc/paligemma

In [ ]:
def process_images(
    images: List[Image.Image],
    size: Dict[str, int] = None,
    resample: Image.Resampling = None,
    rescale_factor: float = None,
    image_mean: Optional[Union[float, List[float]]] = None,
    image_std: Optional[Union[float, List[float]]] = None
) -> List[np.ndarray]:
  height, width = size[0], size[1]
  images = [
      resize(image=image, size=(height, width), resample=resample) for image in images
  ]
  images = [np.array(image) for image in images]
  images = [rescale(image, scale=rescale_factor) for image in images]
  images = [normalize(image, mean=image_mean, std=image_std) for image in images]
  images = [image.transpose(2,0,1) for image in images]
  return images

class PaliGemmaProcessor:

  def __init__(self, tokenizer, num_image_tokens: int, image_size: int):
    super().__init__()
    self.image_seq_length = num_image_tokens
    self.image_size = image_size

    tokens_to_add = {"additional_special_tokens": [self.IMAGE_TOKEN]}
    EXTRA_TOKENS = [
        f"<loc{i:04d}>" for i in range(1024)
    ] # these tokens are used for object detection (bounding boxes)
    EXTRA_TOKENS += [
        f"<seq{i:03d}>" for i in range(128)
    ] # these tokens are used for object segmentation
    tokenizer.add_tokens(EXTRA_TOKENS)
    self.image_token_id = tokenizer.convert_tokens_to_ids(self.IMAGE_TOKENS)
    # We will add the BOS and EOS tokens ourselves
    tokenizer.add_bos_token = False
    tokenizer.add_eos_token = False

    self.tokenizer = tokenizer

  def __call__(
      self,
      text: List[str],
      images: List[Image.Image],
      padding: str = "longest",
      truncation: bool = True
  ) -> dict:
    assert len(images) == 1 and len(text) == 1, f"Received {len(images)} images for {len(text)} prompts."

    pixel_values = process_images(
        images,
        size=(self.image_size, self.image_size),
        resample=Image.Resampling.BICUBIC,
        rescale_factor=1/255.0,
        image_mean=IMAGENET_STANDARD_MEAN,
        image_std=IMAGENET_STANDARD_STD,
    )

    pixel_values = np.stack(pixel_values, axis=0) # (B, channel, height, width)
    pixel_values = torch.tensor(pixel_values)

    input_strings = [
        add_image_tokens_to_prompt(
            prefix_prompt=prompt,
            bos_token=self.tokenizer.bos_token,
            image_seq_len=self.image_seq_length,
            image_token=self.IMAGE_TOKEN
        )
        for prompt in text
    ]

    inputs = self.tokenizer(
        input_strings,
        return_tensors="pt",
        padding=padding,
        truncation=truncation
    )

    return_data = {"pixel_values": pixel_values, **inputs}

    return return_data